In [1]:
from download_sen12 import Pipeline

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
pipe = Pipeline('config.yml', polygons=None, date_range_list=None, dynamic_date_range=True, offset_array=[1])

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [3]:
david__label_path = 'D:/canopy_data/csvs/polygons_101320.csv'

pipe._import_aois(david__label_path)

(<ee.featurecollection.FeatureCollection at 0x2650cc87400>,
 [{'start_date': '2019-12-11',
   'end_date': '2020-03-10',
   'original_date': Timestamp('2020-01-25 00:00:00'),
   'day_offset': 45.0},
  {'start_date': '2019-12-11',
   'end_date': '2020-03-10',
   'original_date': Timestamp('2020-01-25 00:00:00'),
   'day_offset': 45.0},
  {'start_date': '2019-11-30',
   'end_date': '2020-02-28',
   'original_date': Timestamp('2020-01-14 00:00:00'),
   'day_offset': 45.0},
  {'start_date': '2019-11-25',
   'end_date': '2020-02-23',
   'original_date': Timestamp('2020-01-09 00:00:00'),
   'day_offset': 45.0},
  {'start_date': '2019-11-30',
   'end_date': '2020-02-28',
   'original_date': Timestamp('2020-01-14 00:00:00'),
   'day_offset': 45.0},
  {'start_date': '2019-12-11',
   'end_date': '2020-03-10',
   'original_date': Timestamp('2020-01-25 00:00:00'),
   'day_offset': 45.0},
  {'start_date': '2019-11-15',
   'end_date': '2020-02-13',
   'original_date': Timestamp('2019-12-30 00:00:00')

In [4]:
pipe.polygons

In [5]:
pipe.offset_dict

{45: 90, 90: 180, 180: 360}

In [6]:
pipe.run(limit=1)

Processing source 0 with sensor 0
101 features have been loaded
EE directory: COPERNICUS/S2_SR
EE ROI: ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              9.088783,
              5.753152
            ],
            [
              9.179249,
              5.703106
            ],
            [
              9.248257,
              5.792946
            ],
            [
              9.236069,
              5.845887
            ],
            [
              9.090672,
              5.844009
            ],
            [
              9.088783,
              5.753152
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})
start_date: 2019-12-11
end date: 2020-03-10
Filters before: None
Modifiers: [<function inject_B10 at 0x00000153934BF040>, <function sentinel2CloudScore at 0x000001

([[<Task EXPORT_IMAGE: 1_testing_staging_pipeline_Jan_28_14:32:46_2021 (UNSUBMITTED)>]],
 [[]])

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [7]:
pipe.roi_ee

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              9.088783,
              5.753152
            ],
            [
              9.179249,
              5.703106
            ],
            [
              9.248257,
              5.792946
            ],
            [
              9.236069,
              5.845887
            ],
            [
              9.090672,
              5.844009
            ],
            [
              9.088783,
              5.753152
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})

In [8]:
import ee

ee.Initialize()

In [23]:
collection = ee.ImageCollection(pipe.ee_dir) \
                .filterDate(ee.Date(pipe.start_date), ee.Date(pipe.end_date)) \
                .filterBounds(pipe.roi_ee)

In [24]:
collection.size().getInfo()

70

In [16]:
from utils import clipToROI

coll = collection.map( lambda x: clipToROI(x, ee.Geometry(pipe.roi_ee)))

In [17]:
assert pipe.filters_before is not None

AssertionError: 

In [25]:
for m in pipe.modifiers:
    print(m)
    coll = coll.map(m)
    print(coll.size().getInfo())

<function inject_B10 at 0x00000153934BF040>
0
<function sentinel2CloudScore at 0x00000153934B7430>
0
<function calcCloudCoverage at 0x00000153934B7E50>
0
<function sentinel2ProjectShadows at 0x00000153934BF310>
0
<function computeQualityScore at 0x00000153934BF0D0>
0


C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [19]:
coll = coll.filter( pipe.filters_after )

In [20]:
cloudFree = pipe.mergeCollection(coll)

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [21]:
cloudFree.getInfo()

{'type': 'Image', 'bands': []}

In [22]:
coll.size().getInfo()

0

In [27]:
from utils import inject_B10

img = collection.first()

img = inject_B10(img)

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [28]:
img.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32632',
   'crs_transform': [60, 0, 499980, 0, -60, 700020]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 499980, 0, -10, 700020]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,


In [29]:
coll2 = collection.map(inject_B10)

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [30]:
coll2.size().getInfo()

70

In [6]:
pipe.run(limit=1)

Processing source 0 with sensor 0
101 features have been loaded
EE directory: COPERNICUS/S2_SR
EE ROI: ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              9.088783,
              5.753152
            ],
            [
              9.179249,
              5.703106
            ],
            [
              9.248257,
              5.792946
            ],
            [
              9.236069,
              5.845887
            ],
            [
              9.090672,
              5.844009
            ],
            [
              9.088783,
              5.753152
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})
start_date: 2019-12-11
end date: 2020-03-10
Filters before: None
Modifiers: [<function inject_B10 at 0x000002124C998EE0>, <function sentinel2CloudScore at 0x000002

([[None]], [[]])

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [6]:
pipe.run(limit=1)

Processing source 0 with sensor 0
101 features have been loaded
cloudFree info: {'type': 'Image', 'bands': []}


([[<Task EXPORT_IMAGE: 1_testing_staging_pipeline_Jan_28_14:44:14_2021 (UNSUBMITTED)>]],
 [[]])

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [6]:
pipe.run(limit=1)

Processing source 0 with sensor 0
101 features have been loaded
imgC size: 0olygon 1 of 1
best size: 0
Expanding date range
imgC size: 0
best size: 0
Expanding date range
imgC size: 0
best size: 0
Expanding date range
Skipping test
imgC size: 0
best size: 0
best_sorted size: 0
newC size: 2
cloudFree info: {'type': 'Image', 'bands': []}


([[<Task EXPORT_IMAGE: 1_testing_staging_pipeline_Jan_28_14:48:52_2021 (UNSUBMITTED)>]],
 [[]])

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [6]:
pipe.run(limit=1)

Processing source 0 with sensor 0
101 features have been loaded
[<function inject_B10 at 0x0000016636567EE0>, <function sentinel2CloudScore at 0x0000016636567310>, <function calcCloudCoverage at 0x0000016636567D30>, <function sentinel2ProjectShadows at 0x000001663656C1F0>, <function computeQualityScore at 0x0000016636567F70>]
size of collection: 18
Applying modifier <function inject_B10 at 0x0000016636567EE0>
new coll size: 18
Applying modifier <function sentinel2CloudScore at 0x0000016636567310>
new coll size: 18
Applying modifier <function calcCloudCoverage at 0x0000016636567D30>
new coll size: 18
Applying modifier <function sentinel2ProjectShadows at 0x000001663656C1F0>
new coll size: 18
Applying modifier <function computeQualityScore at 0x0000016636567F70>
new coll size: 18
final coll size: 0
imgC size: 0
Expanding date range
[<function inject_B10 at 0x0000016636567EE0>, <function sentinel2CloudScore at 0x0000016636567310>, <function calcCloudCoverage at 0x0000016636567D30>, <funct

([[<Task EXPORT_IMAGE: 1_testing_staging_pipeline_Jan_28_14:51:34_2021 (UNSUBMITTED)>]],
 [[]])

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [5]:
# change config file

exports, exceptions = pipe.run(limit=1)

Processing source 0 with sensor 1
101 features have been loaded


C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
